# Dynamic Routing

How to change how DAC's and DAC's are connected to elements in the Digital Unit Cell

In [1]:
# GRPC boilerplate; enable a connection to the platform and communicate to the UnitCellService

# Generate the protobuf files, if they have not already been generated.
# Genreally, you want to generate them using the `generate_python_interface`
try:
    import qic_unitcell_pb2_grpc as grpc_stub
    import qic_unitcell_pb2 as messages
except ModuleNotFoundError as e:
    # We generate the module on the fly. If the script fails for some reason,
    # another `ModuleNotFoundError` will be thrown 
    import os
    from pathlib import Path
    script_loc = Path.cwd() / '..' / 'generate_python_interface'
    os.system(f'{script_loc} qic_unitcell.proto -- dir {Path.cwd()}')
    
    import qic_unitcell_pb2_grpc as grpc_stub
    import qic_unitcell_pb2 as messages

import grpc    

# The platform to connect to
platform = 'slot3-platform'

conn = grpc.insecure_channel(f'{platform}:50058')

service = grpc_stub.UnitCellServiceStub(conn)

dac_types = {
    0: 'not_connected',
    1: 'manipulation',
    2: 'readout',
    3: 'coupling0',
    4: 'coupling1'
}

def print_dac_signal_types():
    """
    Prints which DAC is connected to which submodule
    """
    res = service.GetDACSignalTypes(messages.Empty())
    print('DAC    |  connection   ')
    print('-------|---------------')
    print('\n'.join([f'{num:<6} | {dac_types[t]}' for num, t in enumerate(res.types)]))

def print_dac_connection():
    """
    Prints which Unit-Cell a DAC is connected to
    """
    print('DAC   | Unit-cells connected to  ')
    print('------|--------------------------')
    for cell_index in range(0, 8):
        routing = service.GetDACRoutingSelect(messages.DACIndex(value=cell_index)).cells
        print(f'{cell_index:<5} | {routing}')

Using `print_dac_signal_types()` we can gather information on which DAC is connected to which submodule. Per default,

- *Manipulation* signals from cells with *even* indices are connected to DACs 0 and 1
- *Readout* signals from cells with *even* indices are connected to DACs 2 and 3
- *Manipulation* signals from cells with *odd* indices are connected to DACs 4 and 5
- *Readout* signals from cells with *odd* indices are connected to DACs 6 and 7

The function uses `GetDACSignalTypes` once to get the connection information 

This is the output for 4 Unit Cells:

DAC    |  connection   
-------|---------------
0      | manipulation
1      | not_connected
2      | readout
3      | not_connected
4      | manipulation
5      | not_connected
6      | readout
7      | not_connected
8      | not_connected
9      | not_connected
10     | not_connected
11     | not_connected
12     | not_connected
13     | not_connected
14     | not_connected
15     | not_connected

Note that the ADC's are real-valued, not complex-valued. # TODO: why?

With `print_dac_connection()` we can see which (complex-valued) DAC is connected to which Unit-Cells.
Using the same setup from above, the following is configured by default:

DAC   | Unit-cells connected to  
------|--------------------------
0     | [0, 2]
1     | []
2     | [0, 2]
3     | []
4     | [1, 3]
5     | []
6     | [1, 3]
7     | []

As an example, let's swap manipulation and readout for the first Unit-Cell

In [2]:
# Route readout from cell 0 to DAC 0
routing = messages.DACRouting(
    type=2, # readout
    dac=0,
    cells=[0]
)

service.SetDACRoutingSelect(routing)

# Route manipulation from cell 0 to DAC 2
routing = messages.DACRouting(
    type=1, # manipulation
    dac=2,
    cells=[0]
)

service.SetDACRoutingSelect(routing)

Nothing changes when using `print_dac_connection()` since cell 0 is still connected to DAC's 0 and 2. However, we can see that using `print_dac_signal_types()` readout and manipulation has swapped

In [3]:
print_dac_signal_types()

DAC    |  connection   
-------|---------------
0      | readout
1      | not_connected
2      | manipulation
3      | not_connected
4      | manipulation
5      | not_connected
6      | readout
7      | not_connected
8      | not_connected
9      | not_connected
10     | not_connected
11     | not_connected
12     | not_connected
13     | not_connected
14     | not_connected
15     | not_connected


Routing ADC's is similar but not as complicated. By default, all even cells are connected to ADC's 4 and 5 (which is ADC 2 when interpreted as one complex ADC) while all odd cells are connected to ADC's 6 and 7 (resp. complex ADC 3).

Using `GetADCRoutingSelect` we can see which cell is connected to which ADC.

In [4]:
service.GetADCRoutingSelect(messages.Empty()).adcs

[2, 3, 2, 3]

The function `GetADCConnectedCells` returns the indices of all Cells that are connected to an ADC

In [6]:
service.GetADCConnectedCells(messages.Empty()).cells

[0, 1, 2, 3]

As an example, we can disconnect ADC 2 from cell 0 and ADC 3 from cell 1

In [7]:
routing = messages.ADCRouting(
    adc=2,
    cells=[0],
    mode=0 # 0: disconnect
)

service.SetADCRoutingSelect(routing)

routing = messages.ADCRouting(
    adc=3,
    cells=[1],
    mode=0 # 0: disconnect
)

service.SetADCRoutingSelect(routing)

We can find that now only cells 2 and 3 are connected.

In [8]:
print(service.GetADCConnectedCells(messages.Empty()).cells)
print(service.GetADCRoutingSelect(messages.Empty()).adcs)

[2, 3]
[2, 3, 2, 3]


Let's now connect ADC 3 to cell 0 and ADC 2 to cell 1

In [9]:
routing = messages.ADCRouting(
    adc=3,
    cells=[0],
    mode=1 # 1: connect
)

service.SetADCRoutingSelect(routing)

routing = messages.ADCRouting(
    adc=2,
    cells=[1],
    mode=1 # 1: connect
)

service.SetADCRoutingSelect(routing)

In [10]:
print(service.GetADCConnectedCells(messages.Empty()).cells)
print(service.GetADCRoutingSelect(messages.Empty()).adcs)

[0, 1, 2, 3]
[3, 2, 2, 3]
